# Dog Image Classification

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

In [3]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.  
We will be using dog classification dataset as example in this project. First of all, we will need to download the data online, and unzip the images. There are three folders consisted here: `test`, `train`, and `valid`. Within each folder, there are 133 folders, where each consists examples of conrresponding dog breed.

In [3]:
#TODO: Fetch and upload the data to AWS S3

# Command to download and unzip data
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!unzip dogImages.zip

--2022-07-12 18:47:56--  https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.192.88
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.192.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1132023110 (1.1G) [application/zip]
Saving to: ‘dogImages.zip.1’

dogImages.zip.1     100%[===================>]   1.05G  25.9MB/s    in 29s     

2022-07-12 18:48:25 (37.6 MB/s) - ‘dogImages.zip.1’ saved [1132023110/1132023110]

Archive:  dogImages.zip
replace dogImages/test/001.Affenpinscher/Affenpinscher_00003.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [4]:
# upload images to s3 bucket
BUCKET = "sagemaker-studio-g0jvojqtzd9"
PREFIX = "dogImages"

sagemaker_session = sagemaker.Session()
# upload_inputs = sagemaker_session.upload_data(path="dogImages", bucket=BUCKET, key_prefix=PREFIX)
upload_inputs = "s3://sagemaker-studio-g0jvojqtzd9/dogImages/"

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [5]:
#TODO: Declare your HP ranges, metrics etc.
# we will need to import some packages to make tuning work
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)


hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch_size": CategoricalParameter([32, 64, 128, 256, 512]),
#     "epochs": IntegerParameter(3, 5),
}

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

In [6]:
#TODO: Create estimators for your HPs
# Import PyTorch, and create estimator along with some parameters such as instance type
from sagemaker.pytorch import PyTorch

ROLE = sagemaker.get_execution_role()

# TODO: Your estimator here
estimator = PyTorch(
    entry_point="hpo.py",
    role=ROLE,
    py_version="py36",
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.m5.large"
)

# TODO: Your HP tuner here
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type=objective_type,
)

In [41]:
# TODO: Fit your HP Tuner
estimator.fit() # TODO: Remember to include your data channels
# tuner.best_training_job()

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: pytorch-training-2022-07-14-07-12-07-525


2022-07-14 07:12:07 Starting - Starting the training job...
2022-07-14 07:12:31 Starting - Preparing the instances for trainingProfilerReport-1657782727: InProgress
.........
2022-07-14 07:13:52 Downloading - Downloading input data...
2022-07-14 07:14:38 Training - Downloading the training image......
2022-07-14 07:15:34 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-07-14 07:15:36,497 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-07-14 07:15:36,500 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-07-14 07:15:36,515 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-07-14 07:15:36,524 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-07-14 07:15:37,086 sagemaker-training-toolkit INFO     N

UnexpectedStatusException: Error for Training job pytorch-training-2022-07-14-07-12-07-525: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python3.6 hpo.py"
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
  0%|          | 0.00/44.7M [00:00<?, ?B/s] 33%|ââââ      | 14.9M/44.7M [00:00<00:00, 156MB/s] 67%|âââââââ   | 29.9M/44.7M [00:00<00:00, 157MB/s]100%|ââââââââââ| 44.7M/44.7M [00:00<00:00, 159MB/s]
Traceback (most recent call last):
  File "hpo.py", line 239, in <module>
    main(args)
  File "hpo.py", line 173, in main
    transform=train_transforms)
  File "/opt/conda/lib/python3.6/site-packages/torchvision/datasets/folder.py", line 256, in __init__
    is_valid_file=is_valid_file)
  File "/opt/conda/lib/python3.6/site-packages/torchvision/datasets/folder.py", line 126, in __init__
    classes, class_to_idx = self._find_cl

In [ ]:
# TODO: Get the best estimators and the best HPs

best_estimator = #TODO

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

estimator = # TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()

In [39]:
#TODO: Import your dependencies.
#For instance, below are some dependencies you might need if you are using Pytorch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

import argparse
import smdebug.pytorch as smd

from PIL import ImageFile






#!!!!!!!!!!!!!!!!!!!!!!!!!!!check normalizer, using gpu, parsing, to device,
#!!!!!!!!!!!!!!!!!!!!!!!!!!!check normalizer, using gpu, parsing, to device,
#!!!!!!!!!!!!!!!!!!!!!!!!!!!check normalizer, using gpu, parsing, to device,
#!!!!!!!!!!!!!!!!!!!!!!!!!!!check normalizer, using gpu, parsing, to device,





NUM_CLASS = 133
TRAIN_DATASET_PATH = "./dogImages/train"
TEST_DATASET_PATH = "./dogImages/test"
VALID_DATASET_PATH = "./dogImages/valid"
MODEL_PATH = "model.pth"


def test(model, test_loader, criterion, device):
    '''
    TODO: Complete this function that can take a model and a 
          testing data loader and will get the test accuray/loss of the model
          Remember to include any debugging/profiling hooks that you might need
    '''
    print("Testing Model on Whole Testing Dataset")
    model.eval() # set the evaluation mode
    running_loss=0
    running_corrects=0
    
    for inputs, labels in test_loader:
        inputs=inputs.to(device)
        labels=labels.to(device)
        outputs=model(inputs)
        loss=criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()

    total_loss = running_loss / len(test_loader.dataset)
    total_acc = running_corrects/ len(test_loader.dataset)
    print(f"Testing Accuracy: {100*total_acc}, Testing Loss: {total_loss}")
    return 100*total_acc



def train(model, train_loader, criterion, optimizer, epochs, device):
    '''
    TODO: Complete this function that can take a model and
          data loaders for training and will get train the model
          Remember to include any debugging/profiling hooks that you might need
    '''

    for epoch in range(1, epochs + 1):
        model.train() # set the training mode
        running_loss = 0.0
        running_corrects = 0
        running_samples=0

        for step, (inputs, labels) in enumerate(train_loader):
            inputs=inputs.to(device)
            labels=labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data).item()
            running_samples+=len(inputs)
            if running_samples % 20  == 0:
                accuracy = running_corrects/running_samples
                print("Images [{}/{} ({:.0f}%)] Loss: {:.2f} Accuracy: {}/{} ({:.2f}%)".format(
                        running_samples,
                        len(train_loader.dataset),
                        100.0 * (running_samples / len(train_loader.dataset)),
                        loss.item(),
                        running_corrects,
                        running_samples,
                        100.0*accuracy,
                    )
                )
                
    return model

    
    
    
def net():
    '''
    TODO: Complete this function that initializes your model
          Remember to use a pretrained model
    '''
    model = models.resnet18(pretrained=True)

    for param in model.parameters():
        param.requires_grad = False   

    num_features=model.fc.in_features
    model.fc = nn.Sequential(
                   nn.Linear(num_features, NUM_CLASS))
    print("next")
    return model



def create_data_loaders(data, batch_size):
    '''
    This is an optional function that you may or may not need to implement
    depending on whether you need to use data loaders or not
    '''
    # data param is assume to be of dataset class
    return torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)



def main(args):
    '''
    TODO: Initialize a model by calling the net function
    '''
    model=net()
    
    '''
    TODO: Create your loss and optimizer
    '''
    loss_criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.fc.parameters(), lr=args.lr)
    
    '''
    TODO: Call the train function to start training your model
    Remember that you will need to set up a way to get training data from S3
    '''
    ## using transform to get data from s3 and convert them into dataloader
    ## class-----------
        ## notice we have downloaded data from online, and unzip the data into 
        ## local folders, so we can directly use them.
    train_transforms = transforms.Compose([  # skipping normalization
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor()
    ])
    
    valid_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
#     ImageFile.LOAD_TRUNCATED_IMAGES = True

    train_dataset = torchvision.datasets.ImageFolder(root=TRAIN_DATASET_PATH,
                                                     transform=train_transforms)
    valid_dataset = torchvision.datasets.ImageFolder(root=VALID_DATASET_PATH,
                                                     transform=valid_transforms)

    # call create_data_loaders method to convert dataset class to dataloader class
    train_loader = create_data_loaders(train_dataset, batch_size=args.batch_size)
    valid_loader = create_data_loaders(valid_dataset, batch_size=args.batch_size)
    

    ## end of converting data--------------
    
    
    if args.gpu == 1:
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    model.to(device)
    
    model=train(model, train_loader, loss_criterion, optimizer, args.epochs, device)
    
    '''
    TODO: Test the model to see its accuracy
    '''
    test(model, valid_loader, loss_criterion, device)
    
    '''
    TODO: Save the trained model
    '''
    torch.save(model, MODEL_PATH)

    
    
if __name__=='__main__':
    print("start")
    parser=argparse.ArgumentParser()
    '''
    TODO: Specify all the hyperparameters you need to use to train your model.
    '''
    parser.add_argument(
        "--batch_size",
        type=int,
        default=64,
        metavar="N",
        help="input batch size for training (default: 64)",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=2,
        metavar="N",
        help="number of epochs to train (default: 10)",
    )
    parser.add_argument(
        "--lr", type=float, default=0.01, metavar="LR", help="learning rate (default: 0.01)"
    )
#     parser.add_argument(
#         "--momentum", type=float, default=0.5, metavar="M", help="SGD momentum (default: 0.5)"
#     )
    parser.add_argument(
        "--gpu", type=int, default=0
    )

    parser.add_argument('-f')
    
    args=parser.parse_args()
    
    main(args)




start
next
Images [320/6680 (5%)] Loss: 8.45 Accuracy: 13/320 (4.06%)


KeyboardInterrupt: 

In [18]:
# !pip install ipywidgets widgetsnbextension pandas-profiling
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import MNIST
from PIL import ImageFile

NUM_CLASS = 133
TRAIN_DATASET_PATH = "../CD0387-deep-learning-topics-within-computer-vision-nlp-project-starter-main/dogImages/train"
TEST_DATASET_PATH = "../dogImages/test"
VALID_DATASET_PATH ="../CD0387-deep-learning-topics-within-computer-vision-nlp-project-starter-main/dogImages/valid"
MODEL_PATH = "model.pth"


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, train_loader, optimizer, epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )


def test(model, test_loader):
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100.0 * correct / len(test_loader.dataset)
        )
    )


def main():
    # Training settings
    parser = argparse.ArgumentParser(description="PyTorch MNIST Example")
    parser.add_argument(
        "--batch-size",
        type=int,
        default=64,
        metavar="N",
        help="input batch size for training (default: 64)",
    )
    parser.add_argument(
        "--test-batch-size",
        type=int,
        default=1000,
        metavar="N",
        help="input batch size for testing (default: 1000)",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=2,
        metavar="N",
        help="number of epochs to train (default: 14)",
    )
    parser.add_argument(
        "--lr", type=float, default=1.0, metavar="LR", help="learning rate (default: 1.0)"
    )
    parser.add_argument("-f")
    args = parser.parse_args()

    train_kwargs = {"batch_size": args.batch_size}
    test_kwargs = {"batch_size": args.test_batch_size}


    
    
    train_transforms = transforms.Compose([  # skipping normalization
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor()
    ])
    
    valid_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    ImageFile.LOAD_TRUNCATED_IMAGES = True

    dataset1 = torchvision.datasets.ImageFolder(root=TRAIN_DATASET_PATH,
                                                     transform=train_transforms)
    dataset2 = torchvision.datasets.ImageFolder(root=VALID_DATASET_PATH,
                                                     transform=valid_transforms)
    
    

    train_loader = torch.utils.data.DataLoader(dataset1, batch_size = 64)
    test_loader = torch.utils.data.DataLoader(dataset2, batch_size=64)

    model = Net()

    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    for epoch in range(1, args.epochs + 1):
        train(model, train_loader, optimizer, epoch)
        test(model, test_loader)
    
    torch.save(model.state_dict(), "mnist_cnn.pt")


if __name__ == "__main__":
    main()



Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[64, 3, 224, 224] to have 1 channels, but got 3 channels instead

In [38]:
model=net()

'''
TODO: Create your loss and optimizer
'''
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=args.lr)

'''
TODO: Call the train function to start training your model
Remember that you will need to set up a way to get training data from S3
'''
## using transform to get data from s3 and convert them into dataloader
## class-----------
    ## notice we have downloaded data from online, and unzip the data into 
    ## local folders, so we can directly use them.
train_transforms = transforms.Compose([  # skipping normalization
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor()
])

valid_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
#     ImageFile.LOAD_TRUNCATED_IMAGES = True

train_dataset = torchvision.datasets.ImageFolder(root=TRAIN_DATASET_PATH,
                                                 transform=train_transforms)
train_loader = create_data_loaders(train_dataset, batch_size=args.batch_size)

next
['_DataLoader__initialized', '_DataLoader__multiprocessing_context', '_IterableDataset_len_called', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_auto_collation', '_dataset_kind', '_get_iterator', '_index_sampler', '_is_protocol', '_iterator', 'batch_sampler', 'batch_size', 'check_worker_number_rationality', 'collate_fn', 'dataset', 'drop_last', 'generator', 'multiprocessing_context', 'num_workers', 'persistent_workers', 'pin_memory', 'prefetch_factor', 'sampler', 'timeout', 'worker_init_fn']
